### Neighborhood enrichment

Computing a neighborhood enrichment can help us identify spots clusters that share a common neighborhood structure across the tissue. We can compute such score with the following function: squidpy.gr.nhood_enrichment(). In short, it’s an enrichment score on spatial proximity of clusters: if spots belonging to two different clusters are often close to each other, then they will have a high score and can be defined as being enriched. On the other hand, if they are far apart, and therefore are seldom a neighborhood, the score will be low and they can be defined as depleted. This score is based on a permutation-based test, and you can set the number of permutations with the n_perms argument (default is 1000).

In [ ]:
from pathlib import Path
import scanpy as sc
import squidpy

import matplotlib.pyplot as plt
DPI = 300
FONTSIZE = 20  # 42
sc.settings.set_figure_params(
    scanpy=True, dpi=100, transparent=True, vector_friendly=True, dpi_save=DPI
)
from matplotlib import rcParams

rcParams["pdf.fonttype"] = 42

from vistools import utils

In [ ]:
SAMPLE_NAME = "concat_withWu2022"

In [ ]:
s = 1.2 # smoothness param from spatialde2

In [ ]:
DIR2LOAD = Path(f"/data/BCI-CRC/nasrine/data/CRC/spatial/CRC_LM_VISIUM/CRC_LM_VISIUM_04_08_09_11/cell2loc_spatialde2/{SAMPLE_NAME}")

In [ ]:
# store results
DIR2SAVE = Path(
    f"/data/BCI-CRC/nasrine/data/CRC/spatial/CRC_LM_VISIUM/CRC_LM_VISIUM_04_08_09_11/neighborhood_enrichment/{SAMPLE_NAME}"
)  # to change
DIR2SAVE.mkdir(parents=True, exist_ok=True)

In [ ]:
# figures
FIG2SAVE = DIR2SAVE.joinpath("figures/")
FIG2SAVE.mkdir(parents=True, exist_ok=True)
# set the global variable: sc.settings.figdir to save all plots
sc.settings.figdir = FIG2SAVE

### Rationale:

I-ve run spatialde2 on cell2loc cell abundance estimates (use the absolute amount of mRNA contributed by each cell type to each spot. We leverage the 5% percentile of the posterior distribution of this parameter (mRNA counts), representing the number of mRNA molecules confidently assigned to each cell type.) over all samples jointly. Observed that some clusters are not seen across all samples. I thought this approach could identify some common clusters (structure) across samples. 

Now I-m looking into building spatial graphs: nodes = spots, edges = neighbourhood relations between spots

it-s a bit tricky when you look at multiple slides at once.

* step 1: create a graph from the spatial coordinates using squidpy.gr.spatial_neighbors but a bit tricky if you have multiple slides as each slide has the same coordinates >> `library_key` in `:func:squidpy.gr.spatial_neighbors` to support building graphs across multiple slides

In [ ]:
cell2loc_counts = sc.read_h5ad(
    DIR2LOAD.joinpath(f"sp_segmentation_smoothness{s}.h5ad")
)

In [ ]:
cell2loc_counts.obs.segmentation_labels.value_counts()

#### add colour to segmentation labels so that it is the same colour for the same label across all visium samples

In [ ]:
cell2loc_counts.uns["segmentation_labels_colors"] = [
    "#fdb462", #0
    "#FCCDE5", #1
    "#FB8072", #2
    "#e78ac3", #3
    "#67A9CF", #4
    "#feed8b", #5
    "#1f78b4", #6
    "#91CF60", #7 
    "#b3ffff", #8 
    "#33a02c" #9
] 

### Idea rename cluster key segmentatio_labels to regions or microenvironments

In [ ]:
cell2loc_counts.obs.rename(columns={"segmentation_labels":"Microenvs"}, inplace=True)

In [ ]:
cell2loc_counts.uns["Microenvs_colors"] = cell2loc_counts.uns["segmentation_labels_colors"]

Since the function works on a connectivity matrix, we need to compute that as well. This can be done with squidpy.gr.spatial_neighbors()

Spatial graph is a graph of spatial neighbors with observations as nodes and neighbor-hood relations between observations as edges. We use spatial coordinates of spots/cells to identify neighbors among them. 

We use squidpy.gr.spatial_neighbors() for this. The function expects coord_type = 'visium' by default. We set this parameter here explicitly for clarity. n_rings should be used only for Visium datasets. It specifies for each spot how many hexagonal rings of spots around will be considered neighbors.

In [ ]:
squidpy.gr.spatial_neighbors(cell2loc_counts, 
                             spatial_key="spatial",
                             n_rings= 4, #2 before we were setting it to 2, 
                             coord_type="grid", 
                             n_neighs=6,
                             library_key="Sample"
                            )

The function builds a spatial graph and saves its adjacency matrix to adata.obsp['spatial_connectivities'] and weighted adjacency matrix to adata.obsp['spatial_distances'] by default. Note that it can also build a a graph from a square grid, just set n_neighs = 4.

**Compute neighborhood enrichment by permutation test.**

A permutation test example: 

do Biology majors spend more on average than sociology majors?

In [ ]:
squidpy.gr.nhood_enrichment(cell2loc_counts,
                            #connectivity_key="spatial_connectivities",
                            cluster_key="Microenvs", #"segmentation_labels", 
                            n_perms=3000,
                            seed=7
                           )

In [ ]:
fig = squidpy.pl.nhood_enrichment(
    cell2loc_counts,
    cluster_key="Microenvs", #"segmentation_labels",
    cmap="coolwarm",
    title="Neighborhood enrichment",
    mode="zscore",
    method="ward",
    dpi=300,
    #figsize=(5, 4),
#     save="nhod_seqfish.png",
    cbar_kwargs={"label": "Z-score"},
    vmin=-10,
    vmax=10,
    n_perms=10000,
    seed=7
)

plt.savefig(
        FIG2SAVE.joinpath(f"{SAMPLE_NAME}_neighbourhood_enrichment_cell2loc_spatialde2_s{s}.pdf"),
        dpi=DPI,
        format="pdf",
        facecolor="w",
        transparent=True,
        edgecolor="w",
        bbox_inches="tight"
    ) 

In [ ]:
fig = squidpy.pl.nhood_enrichment(
    cell2loc_counts,
    cluster_key="Microenvs", #"segmentation_labels",
    cmap="coolwarm",
    title="Neighborhood enrichment",
    mode="zscore",
    method="ward",
    dpi=300,
    #figsize=(5, 4),
#     save="nhod_seqfish.png",
    cbar_kwargs={"label": "Z-score"},
    vmin=-50,
    vmax=50,
    n_perms=10000,
    seed=7
)

plt.savefig(
        FIG2SAVE.joinpath(f"{SAMPLE_NAME}_neighbourhood_enrichment_cell2loc_spatialde2_s{s}_50.pdf"),
        dpi=DPI,
        format="pdf",
        facecolor="w",
        transparent=True,
        edgecolor="w",
        bbox_inches="tight"
    ) 

https://github.com/scverse/squidpy/issues/457

In [ ]:
squidpy.gr.spatial_neighbors(cell2loc_counts, 
                             spatial_key="spatial",
                             n_rings=2,  
                             coord_type="grid", 
                             n_neighs=6,
                             library_key="Sample"
                            )

In [ ]:
squidpy.gr.nhood_enrichment(cell2loc_counts,
                            #connectivity_key="spatial_connectivities",
                            cluster_key="Microenvs", #"segmentation_labels", 
                            n_perms=3000,
                            seed=7
                           )

In [ ]:
fig = squidpy.pl.nhood_enrichment(
    cell2loc_counts,
    cluster_key="Microenvs", #"segmentation_labels",
    cmap="coolwarm",
    title="Neighborhood enrichment",
    mode="zscore",
    method="ward",
    dpi=300,
    #figsize=(5, 4),
#     save="nhod_seqfish.png",
    cbar_kwargs={"label": "Z-score"},
    vmin=-50,
    vmax=50,
    n_perms=3000,
    seed=7
)
